# **Otomoto Data Analysis – Merging and Processing Data**
  
## **1. Importing Libraries**
In this notebook, we will merge scraped CSV files containing vehicle data from Otomoto.  
To accomplish this, we will use the following libraries:

- `pandas` – for data manipulation and working with DataFrames  
- `numpy` – for numerical operations  
- `datetime` – for working with dates and time  

After importing the necessary libraries, we will load the CSV files and merge them into a single dataset, which will then be analyzed.

In [2]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime, timedelta

## **2. Loading Data from CSV Files**
In this section, we load scraped CSV files containing Otomoto listings.  
The data is stored in 24 files named `otomoto_listings_1.csv`, `otomoto_listings_2.csv`, ..., `otomoto_listings_24.csv`.  
Each file is read into a dictionary `dfs`, where the key is `df_{i}`, and the value is the corresponding DataFrame.

Once the data is loaded, we can access specific DataFrames, e.g., `df_1 = dfs['df_1']`, to work with individual datasets.

In [3]:
print("Current working directory:", os.getcwd())
os.chdir(r'..')  # Using a raw string to avoid issues with backslashes
print("Current working directory:", os.getcwd())

Current working directory: c:\Users\Lukasz Pindus\VS Code Python\car_price_analysis\notebooks
Current working directory: c:\Users\Lukasz Pindus\VS Code Python\car_price_analysis


In [4]:
dfs = {}
for i in range(1, 25):
    filename = f'data/otomoto_listings_{i}.csv'
    if os.path.exists(filename):
        dfs[f'df_{i}'] = pd.read_csv(filename)
    else:
        print(f"Warning: {filename} not found!")

## **3. Merging and Preparing Data from Multiple CSV Files**
In this section, we merge all scraped CSV files into a single DataFrame and load an additional dataset, `brands_and_models.csv`, which contains a reference list of vehicle brands and models.

### Steps:
1. Load multiple CSV files containing scraped vehicle listings.
2. Merge them into a single DataFrame using `pd.concat()`, ensuring a continuous index (`ignore_index=True`).
3. Load `brands_and_models.csv` to assist with data validation and standardization.

This step consolidates all available data before performing cleaning and preprocessing.

In [5]:
# Load a reference dataset containing vehicle brands and models
df_brands_models = pd.read_csv('data/brands_and_models.csv')

# Merge all scraped CSV files into a single DataFrame
df = pd.concat(dfs.values(), ignore_index=True)

## **4. Initial Data Exploration**
After merging the data, we perform an initial analysis to understand its structure.  
This includes:
- Displaying the first few rows (`df.head()`)
- Identifying duplicate records (`df[df.duplicated(...)]`)
- Checking column data types (`df.info()`)
- Displaying summary statistics (`df.describe()`)

This helps identify potential issues such as missing values, inconsistencies, or data formatting problems before cleaning.

### **4.1. Preview of the Dataset**
To get an overview of the merged dataset, we display the first few rows.  
This helps us understand the structure of the data and spot any obvious issues.

In [6]:
# Display the first few rows of the merged dataset
df.head()

,ID,Title,Link,Description,Mileage,Fuel Type,Gearbox,Year,Location,Seller Type,Price,Currency,Otomoto Indicator
0,6131159010,Citroën C4 Cactus,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 199 cm3 • 82 KM • Citroen 1.2 82km Polski Sa...,78 000 km,Benzyna,Manualna,2015,Rypin (Kujawsko-pomorskie),Prywatny sprzedawca,38 500,PLN,Powyżej średniej
1,6132197481,Chrysler Pacifica,https://www.otomoto.pl/osobowe/oferta/chrysler...,3 605 cm3 • 288 KM • Limited S design LPG rada...,32 800 km,Benzyna,Automatyczna,2022,Lublin (Lubelskie),Firma,188 000,PLN,NaN
2,6132273174,Citroën C4 PureTech 155 Stop&Start EAT8 SHINE,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 199 cm3 • 155 KM • 2021 Automat Bogate wypos...,19 401 km,Benzyna,Automatyczna,2021,Starogard Gdański (Pomorskie),Prywatny sprzedawca,74 800,PLN,NaN
3,6126251679,Citroën DS4 1.6 THP SportChic,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 598 cm3 • 200 KM • Citroen DS 4,115 700 km,Benzyna,Manualna,2012,Gniezno (Wielkopolskie),Prywatny sprzedawca,30 000,PLN,W granicach średniej
4,6130783778,Chevrolet Captiva 2.2 d LT+,https://www.otomoto.pl/osobowe/oferta/chevrole...,"2 231 cm3 • 183 KM • Stan idealny,nowy akumula...",197 596 km,Diesel,Automatyczna,2012,Białystok (Podlaskie),Prywatny sprzedawca,35 500,PLN,W granicach średniej


### **4.2. Checking for Duplicate Records**
Duplicates can cause data inconsistencies.  
We check for duplicates based on key attributes: `ID`, `Title`, `Link`, and `Description`.

In [7]:
# Identify duplicate records based on key attributes
df[df.duplicated(subset=['ID', 'Title', 'Link', 'Description'], keep=False)]

,ID,Title,Link,Description,Mileage,Fuel Type,Gearbox,Year,Location,Seller Type,Price,Currency,Otomoto Indicator


### **4.3. Checking Data Types and Missing Values**
Before cleaning, it's important to check:
- Column data types (`df.info()`)
- Missing values in key columns

In [8]:
# Display general dataset information (data types, non-null counts)
df.info()

# Count missing values in the 'Link' column
missing_links = df['Link'].isna().sum()
print(f"Number of missing links: {missing_links}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193681 entries, 0 to 193680
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ID                 193681 non-null  int64 
 1   Title              193681 non-null  object
 2   Link               193681 non-null  object
 3   Description        193668 non-null  object
 4   Mileage            193679 non-null  object
 5   Fuel Type          193681 non-null  object
 6   Gearbox            193656 non-null  object
 7   Year               193681 non-null  int64 
 8   Location           193681 non-null  object
 9   Seller Type        193681 non-null  object
 10  Price              193681 non-null  object
 11  Currency           193681 non-null  object
 12  Otomoto Indicator  139847 non-null  object
dtypes: int64(2), object(11)
memory usage: 19.2+ MB
Number of missing links: 0


### **4.4. Counting Unique Listings**
We check the number of unique IDs to see if all listings have distinct identifiers.

In [9]:
# Check the number of unique IDs
print(f"Number of unique IDs: {df['ID'].nunique()}")

Number of unique IDs: 193681


### **4.5. Generating Summary Statistics**
Statistical summaries help us understand price distribution, mileage, and other numerical attributes.

In [10]:
# Generate basic statistical summary
print(df.describe())

                 ID           Year
count  1.936810e+05  193681.000000
mean   6.129758e+09    2015.533780
std    4.114967e+06       6.533054
min    6.003456e+09    1920.000000
25%    6.129279e+09    2011.000000
50%    6.131378e+09    2016.000000
75%    6.131946e+09    2020.000000
max    6.132357e+09    2025.000000


### **4.6. Identifying Unique Rows Based on Key Attributes**
Some duplicate records may not have identical IDs but may still represent the same listing.  
We check for unique listings based on `Title`, `Description`, `Mileage`, `Fuel Type`, `Gearbox`, `Year`, `Location`, `Seller Type`, and `Price`.

In [11]:
# Count unique records based on key listing attributes
unique_rows_count = df.drop_duplicates(subset=['Title', 'Description', 'Mileage', 'Fuel Type', 'Gearbox', 'Year', 'Location', 'Seller Type', 'Price']).shape[0]
print(f"Number of unique rows: {unique_rows_count}")

Number of unique rows: 193681


## **5. Data Cleaning and Standardization**
Before loading the dataset into SQL, we need to clean and standardize the data.  
This step includes:
- **Removing unnecessary or missing values** to ensure data consistency.
- **Standardizing column names** for better readability and uniformity.
- **Converting data types** to facilitate efficient storage and analysis.
- **Normalizing numerical values** to remove unwanted characters like spaces or units.

By cleaning the dataset now, we ensure that it is well-structured and ready for further processing in SQL.

### **5.1. Handling Missing Values**
To ensure data consistency, we remove rows where key attributes are missing.  
The columns we focus on are:
- `Mileage` (should not be null)
- `Price` (should not be null)
- `Gearbox` (essential for categorization)

In [12]:
# Remove rows where key attributes are missing
df.dropna(subset=['Mileage', 'Price', 'Gearbox'], inplace=True)

# Check the dataset info after removing missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 193654 entries, 0 to 193680
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ID                 193654 non-null  int64 
 1   Title              193654 non-null  object
 2   Link               193654 non-null  object
 3   Description        193642 non-null  object
 4   Mileage            193654 non-null  object
 5   Fuel Type          193654 non-null  object
 6   Gearbox            193654 non-null  object
 7   Year               193654 non-null  int64 
 8   Location           193654 non-null  object
 9   Seller Type        193654 non-null  object
 10  Price              193654 non-null  object
 11  Currency           193654 non-null  object
 12  Otomoto Indicator  139840 non-null  object
dtypes: int64(2), object(11)
memory usage: 20.7+ MB


### **5.2. Standardizing Column Names**
For better readability and consistency, we rename some columns:
- `Mileage` → `Mileage in km`

In [13]:
# Rename the 'Mileage' column for clarity
df.rename(columns={'Mileage': 'Mileage in km'}, inplace=True)

### **5.3. Converting Data Types**
To prepare the dataset for SQL storage, we convert string-based numerical columns into proper integer types.
This includes:
- Removing unnecessary spaces and units (e.g., " km", " PLN")
- Converting the values to integer format

In [14]:
# Convert 'Mileage in km' column by removing ' km' and spaces, then casting to integer
df['Mileage in km'] = (
    df['Mileage in km']
    .str.replace(' km', '')
    .str.replace(r'\s+', '', regex=True)
    .str.extract('(\d+)')[0]  # Wyciąga tylko cyfry
    .astype(float)
)

# Convert 'Price' column by removing spaces and casting to integer
df['Price'] = (
    df['Price']
    .str.replace(r'\s+', '', regex=True)
    .str.extract('(\d+)')[0]
    .astype(float)
)

# Check the updated data types
df.dtypes

ID                     int64
Title                 object
Link                  object
Description           object
Mileage in km        float64
Fuel Type             object
Gearbox               object
Year                   int64
Location              object
Seller Type           object
Price                float64
Currency              object
Otomoto Indicator     object
dtype: object

### **5.4. Adding Scraping Date**
Since we are working with scraped data, we add a column to indicate when the data was collected.

In [15]:
# Add a scraping date column (assuming the data is about 31 days old)
df['Scraping Date'] = (datetime.now() - timedelta(days=32)).strftime('%Y-%m-%d')

# Check the updated dataset
df.head()

,ID,Title,Link,Description,Mileage in km,Fuel Type,Gearbox,Year,Location,Seller Type,Price,Currency,Otomoto Indicator,Scraping Date
0,6131159010,Citroën C4 Cactus,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 199 cm3 • 82 KM • Citroen 1.2 82km Polski Sa...,78000.0,Benzyna,Manualna,2015,Rypin (Kujawsko-pomorskie),Prywatny sprzedawca,38500.0,PLN,Powyżej średniej,2025-01-07
1,6132197481,Chrysler Pacifica,https://www.otomoto.pl/osobowe/oferta/chrysler...,3 605 cm3 • 288 KM • Limited S design LPG rada...,32800.0,Benzyna,Automatyczna,2022,Lublin (Lubelskie),Firma,188000.0,PLN,NaN,2025-01-07
2,6132273174,Citroën C4 PureTech 155 Stop&Start EAT8 SHINE,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 199 cm3 • 155 KM • 2021 Automat Bogate wypos...,19401.0,Benzyna,Automatyczna,2021,Starogard Gdański (Pomorskie),Prywatny sprzedawca,74800.0,PLN,NaN,2025-01-07
3,6126251679,Citroën DS4 1.6 THP SportChic,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 598 cm3 • 200 KM • Citroen DS 4,115700.0,Benzyna,Manualna,2012,Gniezno (Wielkopolskie),Prywatny sprzedawca,30000.0,PLN,W granicach średniej,2025-01-07
4,6130783778,Chevrolet Captiva 2.2 d LT+,https://www.otomoto.pl/osobowe/oferta/chevrole...,"2 231 cm3 • 183 KM • Stan idealny,nowy akumula...",197596.0,Diesel,Automatyczna,2012,Białystok (Podlaskie),Prywatny sprzedawca,35500.0,PLN,W granicach średniej,2025-01-07


## **6. Extracting Car Make and Model**
Car make and model are not always stored as separate fields in the dataset.  
Instead, they are usually embedded in the `Title` column.  
To standardize this information, we:
1. Match the make using a predefined reference dataset (`brands_and_models.csv`).
2. Match the model only for rows where the make has already been assigned.

By extracting these values now, we ensure better data consistency before inserting it into SQL.

### **6.1. Extracting Car Make**
We first identify the **car make** by checking whether the `Title` column contains a known brand name.  
This information is extracted using the reference dataset (`brands_and_models.csv`).

In [16]:
# Match car make based on the reference dataset
for _, row in df_brands_models.iterrows():
    make = row['Brand']
    # Assign the make if it appears in the title
    df.loc[df['Title'].str.contains(rf'\b{make}\b', case=False, na=False), 'Make'] = make

### **6.2. Extracting Car Model**
Once the make is assigned, we extract the **model** only for rows where the make has already been determined.  
This ensures that models are assigned correctly to their respective brands.

In [17]:
# Match car model only for rows where the make has already been assigned

# 1. Create a temporary column as a copy of 'Title'
df['Title_temp'] = df['Title']

# 2. Sort models from longest to shortest
df_brands_models_sorted = df_brands_models.copy()
df_brands_models_sorted['model_length'] = df_brands_models_sorted['Model'].apply(len)
df_brands_models_sorted = df_brands_models_sorted.sort_values(by='model_length', ascending=False)

# 3. Iterate through sorted models
for _, row in df_brands_models_sorted.iterrows():
    make = row['Brand']
    model = row['Model']
    
    # Prepare the regex pattern for matching the model name.
    # Using re.escape to prevent issues with special characters in regex.
    pattern = rf'\b{re.escape(model)}\b'
    
    # Create a mask for rows where:
    # - The 'Make' column matches the current brand
    # - The 'Title_temp' column contains the model name (case insensitive)
    mask = (df['Make'] == make) & df['Title_temp'].str.contains(pattern, case=False, na=False)
    
    # Assign the model name to the 'Model' column where the mask is True
    df.loc[mask, 'Model'] = model
    
    # Remove the matched substring from the temporary column to prevent duplicate matches
    df.loc[mask, 'Title_temp'] = df.loc[mask, 'Title_temp'].str.replace(pattern, '', regex=True)
    
    # Optionally, remove extra spaces after substitution
    df.loc[mask, 'Title_temp'] = df.loc[mask, 'Title_temp'].str.strip()

# 4. Remove the temporary column after processing
df.drop(columns=['Title_temp'], inplace=True)


### **6.3. Preview the Extracted Data**
To verify the extraction process, we display the first few rows containing the new `Make` and `Model` columns.

In [18]:
# Display the dataset with extracted make and model
df.head()

,ID,Title,Link,Description,Mileage in km,Fuel Type,Gearbox,Year,Location,Seller Type,Price,Currency,Otomoto Indicator,Scraping Date,Make,Model
0,6131159010,Citroën C4 Cactus,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 199 cm3 • 82 KM • Citroen 1.2 82km Polski Sa...,78000.0,Benzyna,Manualna,2015,Rypin (Kujawsko-pomorskie),Prywatny sprzedawca,38500.0,PLN,Powyżej średniej,2025-01-07,Citroën,C4 Cactus
1,6132197481,Chrysler Pacifica,https://www.otomoto.pl/osobowe/oferta/chrysler...,3 605 cm3 • 288 KM • Limited S design LPG rada...,32800.0,Benzyna,Automatyczna,2022,Lublin (Lubelskie),Firma,188000.0,PLN,NaN,2025-01-07,Chrysler,Pacifica
2,6132273174,Citroën C4 PureTech 155 Stop&Start EAT8 SHINE,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 199 cm3 • 155 KM • 2021 Automat Bogate wypos...,19401.0,Benzyna,Automatyczna,2021,Starogard Gdański (Pomorskie),Prywatny sprzedawca,74800.0,PLN,NaN,2025-01-07,Citroën,C4
3,6126251679,Citroën DS4 1.6 THP SportChic,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 598 cm3 • 200 KM • Citroen DS 4,115700.0,Benzyna,Manualna,2012,Gniezno (Wielkopolskie),Prywatny sprzedawca,30000.0,PLN,W granicach średniej,2025-01-07,Citroën,DS4
4,6130783778,Chevrolet Captiva 2.2 d LT+,https://www.otomoto.pl/osobowe/oferta/chevrole...,"2 231 cm3 • 183 KM • Stan idealny,nowy akumula...",197596.0,Diesel,Automatyczna,2012,Białystok (Podlaskie),Prywatny sprzedawca,35500.0,PLN,W granicach średniej,2025-01-07,Chevrolet,Captiva


## **7. Extracting Engine Capacity and Power**
Car listings often contain engine specifications embedded within the `Description` column.  
To standardize these values, we:
1. Extract **engine capacity (cm³)** by searching for numbers followed by "cm3".
2. Extract **power (KM)** by identifying numbers followed by "KM".
3. Convert these extracted values into numerical format for further analysis.

By performing this step now, we ensure that the dataset is enriched with structured engine data before inserting it into SQL.

### **7.1. Initializing New Columns**
Before extracting values, we create two new columns:
- `Engine capacity` (for engine displacement in cm³)
- `Power` (for engine power in horsepower)

In [19]:
# Initialize new columns
df['Engine capacity'] = None
df['Power'] = None

### **7.2. Extracting Engine Capacity**
We extract engine capacity (cm³) by finding numeric values that appear before "cm3".  
If multiple values exist, we assume the first occurrence represents the engine size.

In [20]:
# Extract engine capacity (cm³) from description
for index, row in df.iterrows():
    description = row['Description']

    # Skip missing descriptions
    if pd.isna(description):
        continue

    # Find engine capacity (number before 'cm3')
    engine_capacity_match = re.search(r'(\d[\d\s]*)cm3', description)
    if engine_capacity_match:
        # Remove spaces inside the number (e.g., '1 397' -> '1397')
        engine_capacity = int(engine_capacity_match.group(1).replace(' ', ''))
        df.at[index, 'Engine capacity'] = engine_capacity

### **7.3. Extracting Engine Power**
We extract power (KM) by searching for numeric values that appear before "KM".  
If multiple values exist, we assume the first occurrence represents the engine power.

In [21]:
# Extract power (KM) from description
for index, row in df.iterrows():
    description = row['Description']

    # Skip missing descriptions
    if pd.isna(description):
        continue

    # Find power (number before 'KM')
    power_match = re.search(r'(\d+)\s*KM', description)
    if power_match:
        power = int(power_match.group(1))
        df.at[index, 'Power'] = power

### **7.4. Converting Data Types**
To ensure proper numerical format, we convert both extracted columns into integers.

In [22]:
# Convert extracted values to integer type
df['Engine capacity'] = df['Engine capacity'].astype('Int64')
df['Power'] = df['Power'].astype('Int64')

In [23]:
# List of numeric columns
numeric_columns = ["ID", "Mileage in km", "Year", "Price", "Engine capacity", "Power"]

# For each numeric column:
for col in numeric_columns:
    # Convert values to numeric; invalid values will become NaN
    df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Print the number of missing/invalid values
    missing_count = df[col].isna().sum()
    print(f"Column '{col}': {missing_count} missing/invalid values")
    
    # replace NaN with None:
    df[col] = df[col].apply(lambda x: None if pd.isna(x) else x)
    
df.replace("", None, inplace=True)
# After this operation, the numeric columns are transformed:
# - All invalid values are replaced with None (which will be interpreted as SQL NULL)
# - Valid values remain unchanged

Column 'ID': 0 missing/invalid values
Column 'Mileage in km': 0 missing/invalid values
Column 'Year': 0 missing/invalid values
Column 'Price': 0 missing/invalid values
Column 'Engine capacity': 3357 missing/invalid values
Column 'Power': 110 missing/invalid values


### **7.5. Previewing the Extracted Data**
To verify the extraction, we display the first few rows, including the new `Engine capacity` and `Power` columns.

In [24]:
# Display the dataset with extracted engine details
df.head()

,ID,Title,Link,Description,Mileage in km,Fuel Type,Gearbox,Year,Location,Seller Type,Price,Currency,Otomoto Indicator,Scraping Date,Make,Model,Engine capacity,Power
0,6131159010,Citroën C4 Cactus,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 199 cm3 • 82 KM • Citroen 1.2 82km Polski Sa...,78000.0,Benzyna,Manualna,2015,Rypin (Kujawsko-pomorskie),Prywatny sprzedawca,38500.0,PLN,Powyżej średniej,2025-01-07,Citroën,C4 Cactus,1199.0,82.0
1,6132197481,Chrysler Pacifica,https://www.otomoto.pl/osobowe/oferta/chrysler...,3 605 cm3 • 288 KM • Limited S design LPG rada...,32800.0,Benzyna,Automatyczna,2022,Lublin (Lubelskie),Firma,188000.0,PLN,NaN,2025-01-07,Chrysler,Pacifica,3605.0,288.0
2,6132273174,Citroën C4 PureTech 155 Stop&Start EAT8 SHINE,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 199 cm3 • 155 KM • 2021 Automat Bogate wypos...,19401.0,Benzyna,Automatyczna,2021,Starogard Gdański (Pomorskie),Prywatny sprzedawca,74800.0,PLN,NaN,2025-01-07,Citroën,C4,1199.0,155.0
3,6126251679,Citroën DS4 1.6 THP SportChic,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 598 cm3 • 200 KM • Citroen DS 4,115700.0,Benzyna,Manualna,2012,Gniezno (Wielkopolskie),Prywatny sprzedawca,30000.0,PLN,W granicach średniej,2025-01-07,Citroën,DS4,1598.0,200.0
4,6130783778,Chevrolet Captiva 2.2 d LT+,https://www.otomoto.pl/osobowe/oferta/chevrole...,"2 231 cm3 • 183 KM • Stan idealny,nowy akumula...",197596.0,Diesel,Automatyczna,2012,Białystok (Podlaskie),Prywatny sprzedawca,35500.0,PLN,W granicach średniej,2025-01-07,Chevrolet,Captiva,2231.0,183.0


### **7.6. Splitting Location into City and Province**  
To improve data structure and facilitate analysis, we split the `Location` column into two new columns:  
- **`City`** → The name of the city  
- **`Province`** → The name of the province  

The city name is extracted from the text before the opening parenthesis, while the province is extracted from the text inside the parentheses. After extraction, the original `Location` column is removed.

In [25]:
# Splitting Location into City and Province

# Check if 'Location' column exists
if "Location" in df.columns:
    # Extract city name (everything before "(")
    df["City"] = df["Location"].str.extract(r"^(.+?) \(")
    
    # Extract province name (everything inside parentheses)
    df["Province"] = df["Location"].str.extract(r"\((.*?)\)")

    # Drop the original 'Location' column
    df.drop(columns=["Location"], inplace=True)

    print("✅ Location column successfully split into 'City' and 'Province'.")

# Display the first rows to verify the transformation
df.head()

✅ Location column successfully split into 'City' and 'Province'.


,ID,Title,Link,Description,Mileage in km,Fuel Type,Gearbox,Year,Seller Type,Price,Currency,Otomoto Indicator,Scraping Date,Make,Model,Engine capacity,Power,City,Province
0,6131159010,Citroën C4 Cactus,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 199 cm3 • 82 KM • Citroen 1.2 82km Polski Sa...,78000.0,Benzyna,Manualna,2015,Prywatny sprzedawca,38500.0,PLN,Powyżej średniej,2025-01-07,Citroën,C4 Cactus,1199.0,82.0,Rypin,Kujawsko-pomorskie
1,6132197481,Chrysler Pacifica,https://www.otomoto.pl/osobowe/oferta/chrysler...,3 605 cm3 • 288 KM • Limited S design LPG rada...,32800.0,Benzyna,Automatyczna,2022,Firma,188000.0,PLN,NaN,2025-01-07,Chrysler,Pacifica,3605.0,288.0,Lublin,Lubelskie
2,6132273174,Citroën C4 PureTech 155 Stop&Start EAT8 SHINE,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 199 cm3 • 155 KM • 2021 Automat Bogate wypos...,19401.0,Benzyna,Automatyczna,2021,Prywatny sprzedawca,74800.0,PLN,NaN,2025-01-07,Citroën,C4,1199.0,155.0,Starogard Gdański,Pomorskie
3,6126251679,Citroën DS4 1.6 THP SportChic,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 598 cm3 • 200 KM • Citroen DS 4,115700.0,Benzyna,Manualna,2012,Prywatny sprzedawca,30000.0,PLN,W granicach średniej,2025-01-07,Citroën,DS4,1598.0,200.0,Gniezno,Wielkopolskie
4,6130783778,Chevrolet Captiva 2.2 d LT+,https://www.otomoto.pl/osobowe/oferta/chevrole...,"2 231 cm3 • 183 KM • Stan idealny,nowy akumula...",197596.0,Diesel,Automatyczna,2012,Prywatny sprzedawca,35500.0,PLN,W granicach średniej,2025-01-07,Chevrolet,Captiva,2231.0,183.0,Białystok,Podlaskie


## **8. Saving Cleaned Data for SQL**
After cleaning and processing the dataset, we save it in a structured format that is ready for SQL ingestion.  
This ensures:
- Data consistency and proper formatting.
- Easier import into SQL without additional preprocessing.
- Better organization for future queries and analysis.

We export the dataset to a CSV file while ensuring that:
1. The file does **not** include the default Pandas index.
2. Column names remain consistent with SQL naming conventions.

### **8.1. Exporting to CSV**
We save the cleaned dataset as a CSV file, ensuring it is properly formatted for SQL.

In [26]:
# Save the cleaned dataset to CSV
df.to_csv('data/cleaned_otomoto_data.csv', index=False)

print("Dataset successfully saved as 'cleaned_otomoto_data.csv'.")

Dataset successfully saved as 'cleaned_otomoto_data.csv'.


### **8.2. Checking the Saved File**
To verify that the export was successful, we reload the file and display its structure.

In [27]:
# Reload the saved file to verify its structure
df_check = pd.read_csv('data/cleaned_otomoto_data.csv')

# Display the first few rows
df_check.head()

,ID,Title,Link,Description,Mileage in km,Fuel Type,Gearbox,Year,Seller Type,Price,Currency,Otomoto Indicator,Scraping Date,Make,Model,Engine capacity,Power,City,Province
0,6131159010,Citroën C4 Cactus,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 199 cm3 • 82 KM • Citroen 1.2 82km Polski Sa...,78000.0,Benzyna,Manualna,2015,Prywatny sprzedawca,38500.0,PLN,Powyżej średniej,2025-01-07,Citroën,C4 Cactus,1199.0,82.0,Rypin,Kujawsko-pomorskie
1,6132197481,Chrysler Pacifica,https://www.otomoto.pl/osobowe/oferta/chrysler...,3 605 cm3 • 288 KM • Limited S design LPG rada...,32800.0,Benzyna,Automatyczna,2022,Firma,188000.0,PLN,NaN,2025-01-07,Chrysler,Pacifica,3605.0,288.0,Lublin,Lubelskie
2,6132273174,Citroën C4 PureTech 155 Stop&Start EAT8 SHINE,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 199 cm3 • 155 KM • 2021 Automat Bogate wypos...,19401.0,Benzyna,Automatyczna,2021,Prywatny sprzedawca,74800.0,PLN,NaN,2025-01-07,Citroën,C4,1199.0,155.0,Starogard Gdański,Pomorskie
3,6126251679,Citroën DS4 1.6 THP SportChic,https://www.otomoto.pl/osobowe/oferta/citroen-...,1 598 cm3 • 200 KM • Citroen DS 4,115700.0,Benzyna,Manualna,2012,Prywatny sprzedawca,30000.0,PLN,W granicach średniej,2025-01-07,Citroën,DS4,1598.0,200.0,Gniezno,Wielkopolskie
4,6130783778,Chevrolet Captiva 2.2 d LT+,https://www.otomoto.pl/osobowe/oferta/chevrole...,"2 231 cm3 • 183 KM • Stan idealny,nowy akumula...",197596.0,Diesel,Automatyczna,2012,Prywatny sprzedawca,35500.0,PLN,W granicach średniej,2025-01-07,Chevrolet,Captiva,2231.0,183.0,Białystok,Podlaskie


In [28]:
df_check.dtypes

ID                     int64
Title                 object
Link                  object
Description           object
Mileage in km        float64
Fuel Type             object
Gearbox               object
Year                   int64
Seller Type           object
Price                float64
Currency              object
Otomoto Indicator     object
Scraping Date         object
Make                  object
Model                 object
Engine capacity      float64
Power                float64
City                  object
Province              object
dtype: object

In [29]:
df_check.count()

ID                   193654
Title                193654
Link                 193654
Description          193642
Mileage in km        193654
Fuel Type            193654
Gearbox              193654
Year                 193654
Seller Type          193654
Price                193654
Currency             193654
Otomoto Indicator    139840
Scraping Date        193654
Make                 193653
Model                193009
Engine capacity      190297
Power                193544
City                 193654
Province             193654
dtype: int64